In [ ]:

import os

import numpy as np
import tifffile as tiff
from tqdm import trange

data_path = '/share/data/CryoET_Data/lizhuo/data_for_liushuo/3D_training_data/mask5/l5-actin-1.tif'
mask_sign = '_spine_result'
mask_path = data_path.replace('.tif',mask_sign+'.tif')
file_name = data_path.split('/')[-1].split('.')[0]
mask_name = mask_path.split('/')[-1].split('.')[0]
save_path = os.path.join(data_path.split(file_name)[0],'patchs')
if not os.path.exists(save_path):
    os.makedirs(save_path)
patch_size = 1024
overlap_rate = 0.5




def equalize_histogram_sqrt(image):

    img_np = np.asarray(image)

    # 计算原始直方图
    histogram, bin_edges = np.histogram(img_np.flatten(), bins=256, range=(0, 256))

    # 对直方图的值取平方根
    histogram_sqrt = np.sqrt(histogram)

    # 计算累计分布函数 (CDF)
    cdf = histogram_sqrt.cumsum()
    cdf_normalized = 255 * (cdf - cdf.min()) / (cdf.max() - cdf.min())
    cdf_normalized = cdf_normalized.astype('uint8')

    # 使用平方根处理后的 CDF 映射像素值
    equalized_img_np = cdf_normalized[img_np]

    # 将均衡化后的数组转换回图像
    #equalized_image_pil = Image.fromarray(equalized_img_np)

    return equalized_img_np

def image_preprocess(img, percentile=99.5,equalize=False):
#input: PIL image or np array
#max percentile + normalize -> 0-255 int(uint8)
#output: np array
    if type(img) is np.ndarray:
        img_np = img
    else:
        img_np = np.array(img)
        #print("warning: input image is not np.ndarray")

    max_percentile = np.percentile(img_np, percentile)
    img_np = np.minimum(img_np, max_percentile)
    img_np = (img_np - np.min(img_np)) / (np.max(img_np) - np.min(img_np))
    img_np = img_np * 255
    img_np = img_np.astype(np.uint8)

    if equalize:
        img_np = equalize_histogram_sqrt(img_np)

    return img_np

def get_patch(data, patch_size, overlap_rate):
    data_patch = []
    for i in range(0, data.shape[1]-int(patch_size*(1-overlap_rate)), int(patch_size*(1-overlap_rate))):
        for j in range(0, data.shape[2]-int(patch_size*(1-overlap_rate)), int(patch_size*(1-overlap_rate))):
            data_patch.append(data[:,i:i+patch_size,j:j+patch_size])
    return data_patch


data_exists = os.path.exists(data_path)
mask_exists = os.path.exists(mask_path)
if data_exists:
    data = tiff.imread(data_path)
    if len(data.shape) == 2:
        data = data[np.newaxis,...]
    print('processing data with shape:', data.shape)
    processed_data = np.zeros(data.shape)
    for t in trange(data.shape[0]):
        processed_data[t] = image_preprocess(data[t])
    print('getting patchs...')
    data_patch = get_patch(processed_data, patch_size, overlap_rate)
    print('saving patchs...')
    for i in trange(len(data_patch)):
        tiff.imwrite(os.path.join(save_path, file_name+'-'+f"{i:04d}"+'.tif'), data_patch[i].astype(np.uint8))
if mask_exists:
    mask = tiff.imread(mask_path)
    if len(mask.shape) == 2:
        mask = mask[np.newaxis,...]
    print('processing mask with shape:', mask.shape)
    print('getting patchs...')
    mask_patch = get_patch(mask, patch_size, overlap_rate)
    print('saving patchs...')
    for i in trange(len(data_patch)):
        tiff.imwrite(os.path.join(save_path, mask_name+'-'+f"{i:04d}"+'.tif'), mask_patch[i].astype(np.uint16))

print('done!')


In [ ]:
import os

import numpy as np
import tifffile as tiff
from scipy.ndimage import find_objects, label
from tqdm import trange

#对文件夹下所有文件处理版

# 设置文件夹路径
data_folder = '/share/data/CryoET_Data/lizhuo/data_for_liushuo/3D_data_with_mask/l5'  # 文件夹路径
mask_signs = ('_spine_result','-hwl-spine')
patch_size = 1024
overlap_rate = 0.5

# 获取文件夹中所有的 .tif 文件
tif_files = [f for f in os.listdir(data_folder) if f.endswith('.tif')]

def equalize_histogram_sqrt(image):
    img_np = np.asarray(image)
    # 计算原始直方图
    histogram, bin_edges = np.histogram(img_np.flatten(), bins=256, range=(0, 256))

    # 对直方图的值取平方根
    histogram_sqrt = np.sqrt(histogram)

    # 计算累计分布函数 (CDF)
    cdf = histogram_sqrt.cumsum()
    cdf_normalized = 255 * (cdf - cdf.min()) / (cdf.max() - cdf.min())
    cdf_normalized = cdf_normalized.astype('uint8')

    # 使用平方根处理后的 CDF 映射像素值
    equalized_img_np = cdf_normalized[img_np]

    return equalized_img_np

def image_preprocess(img, percentile=99.5, equalize=False):
    # input: PIL image or np array
    # max percentile + normalize -> 0-255 int(uint8)
    # output: np array
    if type(img) is np.ndarray:
        img_np = img
    else:
        img_np = np.array(img)

    max_percentile = np.percentile(img_np, percentile)
    img_np = np.minimum(img_np, max_percentile)
    img_np = (img_np - np.min(img_np)) / (np.max(img_np) - np.min(img_np))
    img_np = img_np * 255
    img_np = img_np.astype(np.uint8)

    if equalize:
        img_np = equalize_histogram_sqrt(img_np)

    return img_np



def filter_small(img, threshold):
    result = np.array(img)
    if len(result.shape) == 2:
        # 使用 SciPy 的 label 函数标记连通区域
        labeled_image, num_features = label(result)

        # 获取每个区域的边界框
        objects = find_objects(labeled_image)

        for i, obj in enumerate(objects, start=1):
            area = np.sum(labeled_image[obj] == i)
            if area < threshold:
                labeled_image[labeled_image == i] = 0
            else:
                labeled_image[labeled_image == i] = i

        result = labeled_image
    else:
        for t in range(result.shape[0]):
            labeled_image, num_features = label(result[t])
            objects = find_objects(labeled_image)
            for i, obj in enumerate(objects, start=1):
                area = np.sum(labeled_image[obj] == i)
                if area < threshold:
                    labeled_image[labeled_image == i] = 0
                else:
                    labeled_image[labeled_image == i] = i
            result[t] = labeled_image

    return result

def reset_id(segmentation):
    # 获取唯一值
    values = np.unique(segmentation)

    # 检查是否需要重置 ID
    if np.max(segmentation) == len(values) - 1:
        return segmentation

    # 创建新的分割 ID 数组
    reset_id_seg = np.zeros_like(segmentation, dtype=int)
    # 过滤掉0值
    non_zero_values = values[values != 0]

    # 使用 np.arange 生成新 ID
    reset_id_seg[np.isin(segmentation, non_zero_values)] = np.arange(1, len(non_zero_values) + 1)[np.searchsorted(non_zero_values, segmentation[segmentation != 0])]

    return reset_id_seg.astype(np.uint16)

def get_patch(data, patch_size, overlap_rate):
    data_patch = []
    for i in range(0, data.shape[1] - int(patch_size * (1 - overlap_rate)), int(patch_size * (1 - overlap_rate))):
        for j in range(0, data.shape[2] - int(patch_size * (1 - overlap_rate)), int(patch_size * (1 - overlap_rate))):
            data_patch.append(data[:, i:i + patch_size, j:j + patch_size])
    return data_patch


# 遍历文件夹中的所有 .tif 文件
for tif_file in tif_files:
    print(f"Processing file: {tif_file}")

    data_path = os.path.join(data_folder, tif_file)
    file_name = tif_file.split('.')[0]
    save_path = os.path.join(data_folder, 'patches')

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    if file_name.endswith(mask_signs):
        data_type = 'mask'
    else:
        data_type = 'data'

    if data_type == 'data':
        data = tiff.imread(data_path)
        if len(data.shape) == 2:
            data = data[np.newaxis, ...]
        print(f'Processing data with shape: {data.shape}')
        processed_data = np.zeros(data.shape)
        for t in trange(data.shape[0]):
            processed_data[t] = image_preprocess(data[t])
        print('Getting patches for data...')
        data_patch = get_patch(processed_data, patch_size, overlap_rate)
        print('Saving data patches...')
        for i in trange(len(data_patch)):
            patch_file = os.path.join(save_path, f"{file_name}-{i:04d}.tif")
            tiff.imwrite(patch_file, data_patch[i].astype(np.uint8))

    if data_type == 'mask':
        mask = tiff.imread(data_path)
        if len(mask.shape) == 2:
            mask = mask[np.newaxis, ...]
        print(f'Reset id with mask shape: {mask.shape}')
        mask = filter_small(mask,0)
        print('Getting patches for mask...')
        mask_patch = get_patch(mask, patch_size, overlap_rate)
        print('Saving mask patches...')
        for i in trange(len(mask_patch)):
            patch_file = os.path.join(save_path, f"{file_name}-{i:04d}.tif")
            tiff.imwrite(patch_file, mask_patch[i].astype(np.uint16))




print('All files processed!')


In [39]:
import os

import numpy as np
import tifffile as tiff

#from scipy.ndimage.filter import gaussian_filter
from natsort import natsorted
from tqdm import trange

# def get_gaussian(s, sigma=4.0/8) -> np.ndarray:
# 	temp = np.zeros(s)
# 	coords = [i // 2 for i in s]
# 	sigmas = [i * sigma for i in s]
# 	temp[tuple(coords)] = 1
# 	gaussian_map = gaussian_filter(temp, sigmas, 0, mode='constant', cval=0)
# 	gaussian_map /= np.max(gaussian_map)
# 	return gaussian_map.astype(np.float32)

def patches2image(patches,overlap_rate,sigma=4.0/8):
    #不适合
    if patches[0].ndim == 2:
        patches = np.array(patches)[np.newaxis,...]
    patch_t = patches[0].shape[0]
    patch_size = patches[0].shape[1] #1024
    stride = int(patch_size * (1 - overlap_rate)) #512
    patch_num = int(np.ceil(np.sqrt(len(patches)))) #5
    image_size = (patch_num+1) * stride # 3072
    image = np.zeros((patch_t,image_size, image_size), dtype=np.float32)
    normal_map = np.zeros((patch_t,image_size, image_size), dtype=np.float32)
    gaussian_map = get_gaussian((patch_size, patch_size), sigma)
    gaussian_map = np.tile(gaussian_map[np.newaxis,...],(patch_t,1,1))

    gaussian_map = np.ones_like(gaussian_map)

    for i in range(patch_num):
        for j in range(patch_num):
            patch = patches[i * patch_num + j]
            image[:,i * stride:i * stride + patch_size, j * stride:j * stride + patch_size] += patch * gaussian_map
            normal_map[:,i * stride:i * stride + patch_size, j * stride:j * stride + patch_size] += gaussian_map
    image = image / normal_map

    return image.astype(np.uint16)

def patches2image2(patches,overlap_rate):
    # 小瑕疵
    if patches[0].ndim == 2:
        patches = np.array(patches)[np.newaxis,...]
    patch_t = patches[0].shape[0]
    patch_size = patches[0].shape[1] #1024
    stride = int(patch_size * (1 - overlap_rate)) #512
    patch_num = int(np.ceil(np.sqrt(len(patches)))) #5
    image_size = (patch_num+1) * stride # 3072
    image = np.zeros((patch_t,image_size, image_size), dtype=np.float32)
    count_map = np.zeros((patch_t,image_size, image_size), dtype=np.float32)


    for i in trange(patch_num):
        for j in trange(patch_num):
            patch = patches[i * patch_num + j]
            image[:,i * stride:i * stride + patch_size, j * stride:j * stride + patch_size] += patch
            count_map[:,i * stride:i * stride + patch_size, j * stride:j * stride + patch_size] += np.where(patch>0,1,0)
    count_map[count_map==1] = 0
    count_map[count_map==2] = 1/2
    count_map[count_map==3] = 1/3
    count_map[count_map==4] = 1/4
    image = image * count_map

    return image.astype(np.uint16)

def patches2image3(patches,overlap_rate):
    #崩溃
    print('patches2image ing...')
    if patches[0].ndim == 2:
        patches = np.array(patches)[np.newaxis,...]
    patch_t = patches[0].shape[0]
    patch_size = patches[0].shape[1] #1024
    stride = int(patch_size * (1 - overlap_rate)) #512
    patch_num = int(np.ceil(np.sqrt(len(patches)))) #5
    image_size = (patch_num+1) * stride # 3072

    # image = np.zeros((patch_t,image_size, image_size), dtype=np.float32)
    # #image的每一个值是一个列表，是所有patch在该位置的值的列表
    # for i in trange(patch_num):
    #     for j in trange(patch_num):
    #         patch = patches[i * patch_num + j]
    #         #把patch的值append到image的对应位置的list
    #         image[:,i * stride:i * stride + patch_size, j * stride:j * stride + patch_size].append(patch)
    # #对image的每一个列表取众数
    # for i in trange(image.shape[0]):
    #     for j in trange(image.shape[1]):
    #         for k in trange(image.shape[2]):
    #             image[i,j,k] = np.argmax(np.bincount(image[i,j,k]))

    # 维度是 (t, image_size, image_size)，每个位置是一个列表，用来存储该位置的所有patch
    image = np.empty((patch_t, image_size, image_size), dtype=object)
    print('image.shape:',image.shape)
    # 初始化每个位置的列表
    for t in range(patch_t):
        for i in range(image_size):
            for j in range(image_size):
                image[t, i, j] = []

    # 将patch添加到对应位置的列表中
    for idx in trange(len(patches)):
        i = idx // patch_num  # 计算行号
        j = idx % patch_num   # 计算列号
        patch = patches[idx]


        for t in range(patch_t):
            for x in range(patch_size):
                for y in range(patch_size):
                    image[t, i * stride + x, j * stride + y].append(patch[t, x, y])

    # # 计算每个位置的众数
    # for i in trange(image.shape[1]):
    #     for j in trange(image.shape[2]):
    #         for t in range(patch_t):
    #             # 获取当前位置的所有patch，并计算众数
    #             patch_list = image[t, i, j]
    #             if patch_list:
    #                 image[t, i, j] = np.argmax(np.bincount(patch_list))

    return image.astype(np.uint16)

def patches2image4(patches,overlap_rate):
    #慢？
    print('patches2image ing...')
    if patches[0].ndim == 2:
        patches = np.array(patches)[np.newaxis,...]
    patch_t = patches[0].shape[0]
    patch_size = patches[0].shape[1] #1024
    stride = int(patch_size * (1 - overlap_rate)) #512
    patch_num = int(np.ceil(np.sqrt(len(patches)))) #5
    image_size = (patch_num+1) * stride # 3072

    group = [0,1,0,1,0,
             2,3,2,3,2,
             0,1,0,1,0,
             2,3,2,3,2,
             0,1,0,1,0]

    image = np.zeros((patch_t,image_size, image_size), dtype=np.uint16)
    for t in trange(patch_t):
        empty_image_4 = np.ones((image_size, image_size,4), dtype=np.uint16) * (-1)
        image_t = np.zeros((image_size, image_size), dtype=np.uint16)
        for i in range(patch_num):
            for j in range(patch_num):
                patch = patches[i * patch_num + j][t]
                empty_image_4[i * stride:i * stride + patch_size, j * stride:j * stride + patch_size, group[i*patch_num + j]] = patch
        for i in range(image_size):
            for j in range(image_size):
                #排除-1
                image_t[i,j] = np.argmax(np.bincount(empty_image_4[i,j][empty_image_4[i,j] != -1]))


        image[t] = image_t
    return image.astype(np.uint16)


In [10]:
import os

import numpy as np
import tifffile as tiff

patches_path = '/share/data/CryoET_Data/lizhuo/data_for_liushuo/3D_data_with_mask/l1/patches/results_mapping'
save_path = patches_path.split('patches')[0]
patch_files = [f for f in os.listdir(patches_path) if f.endswith('.tif')]
patch_files = natsorted(patch_files)
patches = []
for patch_file in patch_files:
    print(f"Processing patch: {patch_file}")
    patch = tiff.imread(os.path.join(patches_path, patch_file))
    patches.append(patch)


Processing patch: l1-actin-0000_result.tif
Processing patch: l1-actin-0001_result.tif
Processing patch: l1-actin-0002_result.tif
Processing patch: l1-actin-0003_result.tif
Processing patch: l1-actin-0004_result.tif
Processing patch: l1-actin-0005_result.tif
Processing patch: l1-actin-0006_result.tif
Processing patch: l1-actin-0007_result.tif
Processing patch: l1-actin-0008_result.tif
Processing patch: l1-actin-0009_result.tif
Processing patch: l1-actin-0010_result.tif
Processing patch: l1-actin-0011_result.tif
Processing patch: l1-actin-0012_result.tif
Processing patch: l1-actin-0013_result.tif
Processing patch: l1-actin-0014_result.tif
Processing patch: l1-actin-0015_result.tif
Processing patch: l1-actin-0016_result.tif
Processing patch: l1-actin-0017_result.tif
Processing patch: l1-actin-0018_result.tif
Processing patch: l1-actin-0019_result.tif
Processing patch: l1-actin-0020_result.tif
Processing patch: l1-actin-0021_result.tif
Processing patch: l1-actin-0022_result.tif
Processing 

In [40]:
empty_image = patches2image4(patches,0.5)
tiff.imwrite(os.path.join(save_path, 'result4.tif'), empty_image)

patches2image ing...


100%|██████████| 80/80 [22:47<00:00, 17.09s/it]


In [27]:
empty_image[:,:,1]

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int32)

In [4]:
#image = patches2image(patches,overlap_rate = 0.5)
image = patches2image3(patches,overlap_rate = 0.5)
tiff.imwrite(os.path.join(save_path, 'result3.tif'), image)

  4%|▍         | 1/25 [00:11<04:37, 11.56s/it]

: 

In [4]:
patches_path = '/share/data/CryoET_Data/lizhuo/data_for_liushuo/3D_data_with_mask/1/patches/results_mapping'
save_path = patches_path.split('patches')[0]
patch_files = [f for f in os.listdir(patches_path) if f.endswith('.tif')]
patch_files = natsorted(patch_files)
patches = []
for patch_file in patch_files:
    patch = tiff.imread(os.path.join(patches_path, patch_file))
    patches.append(patch)
image = patches2image2(patches,overlap_rate = 0.5)
tiff.imwrite(os.path.join(save_path, 'result2.tif'), image)

100%|██████████| 5/5 [00:09<00:00,  1.87s/it]


In [16]:
from itertools import chain

list1 = [[1, 1], 2]
list2 = [3, 4]

# 使用zip将两个列表逐个合并
merged_list = [list(chain(a, [b])) for a, b in zip(list1, list2)]

print(merged_list)

TypeError: 'int' object is not iterable

In [19]:
print(zip(list1, list2)[0])

TypeError: 'zip' object is not subscriptable

In [9]:
a[0].append(3)

In [25]:
a = [[],[]]
a.append(3)
a

[[], [], 3]

In [30]:
image = np.empty((5,3 ,3 ), dtype=object)

In [35]:
image[0][0][0] = []

In [36]:
image

array([[[list([]), None, None],
        [None, None, None],
        [None, None, None]],

       [[None, None, None],
        [None, None, None],
        [None, None, None]],

       [[None, None, None],
        [None, None, None],
        [None, None, None]],

       [[None, None, None],
        [None, None, None],
        [None, None, None]],

       [[None, None, None],
        [None, None, None],
        [None, None, None]]], dtype=object)

In [7]:
import numpy as np
import tifffile as tiff

patch_label = np.zeros((1,3072,3072),dtype=np.uint16)
for i in range(25):
    row = i // 5
    col = i % 5
    patch_label[0,row*512+10:row*512+1014,col*512+10:col*512+10+i] = i
    patch_label[0,row*512+10:row*512+1014,col*512+1014:col*512+1014-i] = i
    patch_label[0,row*512+10:row*512+10+i,col*512+10:col*512+1014] = i
    patch_label[0,row*512+1014:row*512+1014-i,col*512+10:col*512+1014] = i

tiff.imwrite('/share/data/CryoET_Data/lizhuo/data_for_liushuo/3D_data_with_mask/patches_label.tif',patch_label)


In [8]:
patch

NameError: name 'patch' is not defined